In [1]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

np.random.seed(40)
warnings.filterwarnings("ignore")

# Setup Experiment Tracker

In [2]:
tracking_uri='file:///mnt/pipelineai/users/experiments'
mlflow.set_tracking_uri(tracking_uri)

experiment_name = 'wine'
mlflow.set_experiment(experiment_name)    

# Import Training Data

In [3]:
# Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
wine_path = './datasets/wine-quality.csv'
data = pd.read_csv(wine_path)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# Start Training Run

In [4]:
alpha = 0.20
l1_ratio = 0.20

In [5]:
# Parameters
alpha = 0.6
ratio = 0.1


In [6]:
with mlflow.start_run() as run:
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # Log Parameters
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)

    # Log Metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    # Log Model
    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.600000, l1_ratio=0.200000):
  RMSE: 0.8052136137150304
  MAE: 0.622535932743478
  R2: 0.1625823887208505


# Check the MLflow Pipelines Tab
![MLflow Pipeline](https://raw.githubusercontent.com/PipelineAI/site/master/assets/img/mlflow-pipeline.png)